<a href="https://colab.research.google.com/github/RROS-Lab/robot_surface_heating/blob/learning_k_vals/K_value_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.optimize import curve_fit, minimize, least_squares
from numpy import genfromtxt
import numpy as np
import copy
from matplotlib import pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataPath = "drive/My Drive/Heat Planning/flat_3_no_cooling.csv"
dataPath1 = "drive/My Drive/Heat Planning/Training_Data/fullheating.csv"
dataPath2 = "drive/My Drive/Heat Planning/Training_Data/node4.csv"
dataPath3 = "drive/My Drive/Heat Planning/Training_Data/node5.csv"
dataPath4 = "drive/My Drive/Heat Planning/Training_Data/node7.csv"

testdataPath1 = "drive/My Drive/Heat Planning/Testing_Data/node5.csv"
testdataPath2 = "drive/My Drive/Heat Planning/Testing_Data/node6.csv"
testdataPath3 = "drive/My Drive/Heat Planning/Testing_Data/node9.csv"


Mounted at /content/drive


In [ ]:
def thermal_func(input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,Q9 = input
  global time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input
  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9]]
  Q=[[0],[0],[0],[0],[0],[0],[0],[0],[Q9]]
  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9]])
  k_mat = [[k12+k14+ke1, -k12, 0, -k14, 0, 0, 0, 0, 0],\
      [-k12, k12+k25+k23+ke2, -k23, 0, -k25, 0, 0, 0, 0],\
      [0, -k23, k23+k36+ke3, 0, 0, -k36, 0, 0, 0],\
      [-k14, 0, 0, k14+k45+k47+ke4, -k45, 0, -k47, 0, 0],\
      [0, -k25, 0, -k45, k25+k45+k56+k58+ke5, -k56, 0,  -k58, 0],\
      [0, 0, -k36, 0, -k56, k36+k56+k69+ke6, 0, 0, -k69],\
      [0, 0, 0, -k47, 0, 0, k47+k78+ke7, -k78, 0],\
      [0, 0, 0, 0, -k58, 0, -k78, k58+k78+k89+ke8, -k89],\
      [0,   0,   0,   0,   0,   -k69, 0, -k89, k69+k89+ke9]]

  Temp = copy.deepcopy(T)

  for x in range(len(time)-1):
    i = x+1
    # T_Q9+ke9*72-(0*T1+0*T2+0*T3+0*T4+0*T5-k69*T6+0*T7-k89*T8+(k69+k89+ke9)*T9[i])*(time(i)-time(i-1))-T9(i-1)
    T=(Q+ke*72 - np.matmul(k_mat,T))*(time[i]-time[i-1]) + T
    Temp = np.concatenate((Temp,T),1)
  Temp_gen = Temp[8,:]
  error = abs(np.transpose(Temp)-T_input)
  error=error.sum()/len(T_input[:,1])
  # print(error)
  # print(len(T_input[:,1]))
  return error

In [ ]:
def thermal_func2(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,Q9 = input
  # time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input = *args

  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9]]

  # Q=[[0],[0],[0],[0],[0],[0],[0],[0],[Q9]]
  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9]])
  k_mat = [[k12+k14+ke1, -k12, 0, -k14, 0, 0, 0, 0, 0],\
      [-k12, k12+k25+k23+ke2, -k23, 0, -k25, 0, 0, 0, 0],\
      [0, -k23, k23+k36+ke3, 0, 0, -k36, 0, 0, 0],\
      [-k14, 0, 0, k14+k45+k47+ke4, -k45, 0, -k47, 0, 0],\
      [0, -k25, 0, -k45, k25+k45+k56+k58+ke5, -k56, 0,  -k58, 0],\
      [0, 0, -k36, 0, -k56, k36+k56+k69+ke6, 0, 0, -k69],\
      [0, 0, 0, -k47, 0, 0, k47+k78+ke7, -k78, 0],\
      [0, 0, 0, 0, -k58, 0, -k78, k58+k78+k89+ke8, -k89],\
      [0,   0,   0,   0,   0,   -k69, 0, -k89, k69+k89+ke9]]

  Temp = copy.deepcopy(T)

  for x in range(len(time)-1):
    i = x+1
    # T_Q9+ke9*72-(0*T1+0*T2+0*T3+0*T4+0*T5-k69*T6+0*T7-k89*T8+(k69+k89+ke9)*T9[i])*(time(i)-time(i-1))-T9(i-1)
    T=(Q+ke*100 - np.matmul(k_mat,T))*(time[i]-time[i-1]) + T
    Temp = np.concatenate((Temp,T),1)
  Temp_gen = Temp[8,:]
  error = sum(sum(abs(np.transpose(Temp)-T_input)))/len(Temp[:,1])
  print(error)
  return error

In [ ]:
def thermal_sim(input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,Q9 = input
  global time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input
  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9]]
  Q=[[0],[0],[0],[0],[0],[0],[0],[0],[Q9]]
  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9]])
  k_mat = [[k12+k14+ke1, -k12, 0, -k14, 0, 0, 0, 0, 0],\
      [-k12, k12+k25+k23+ke2, -k23, 0, -k25, 0, 0, 0, 0],\
      [0, -k23, k23+k36+ke3, 0, 0, -k36, 0, 0, 0],\
      [-k14, 0, 0, k14+k45+k47+ke4, -k45, 0, -k47, 0, 0],\
      [0, -k25, 0, -k45, k25+k45+k56+k58+ke5, -k56, 0,  -k58, 0],\
      [0, 0, -k36, 0, -k56, k36+k56+k69+ke6, 0, 0, -k69],\
      [0, 0, 0, -k47, 0, 0, k47+k78+ke7, -k78, 0],\
      [0, 0, 0, 0, -k58, 0, -k78, k58+k78+k89+ke8, -k89],\
      [0,   0,   0,   0,   0,   -k69, 0, -k89, k69+k89+ke9]]

  Temp = copy.deepcopy(T)

  for x in range(len(time)-1):
    i = x+1
    # T_Q9+ke9*72-(0*T1+0*T2+0*T3+0*T4+0*T5-k69*T6+0*T7-k89*T8+(k69+k89+ke9)*T9[i])*(time(i)-time(i-1))-T9(i-1)
    T=(Q+ke*72 - np.matmul(k_mat,T))*(time[i]-time[i-1]) + T
    Temp = np.concatenate((Temp,T),1)
  Temp_gen = Temp[8,:]
  # print(Temp_gen-T_input)
  # error = sum(sum(abs(Temp_gen-T_input)))
  return Temp_gen,error, Temp_gen-T_input

In [ ]:
# my_data = genfromtxt(dataPath, delimiter=',')
# time = my_data[:,0]
# T1=my_data[0,1]
# T2=my_data[0,2]
# T3=my_data[0,3]
# T4=my_data[0,4]
# T5=my_data[0,5]
# T6=my_data[0,6]
# T7=my_data[0,7]
# T8=my_data[0,8]
# T9=my_data[0,9]
# global time,T1,T2,T3,T4,T5,T6,T7,T8,T9
# global T_input

# # T_input= my_data[0:70,9]
# T_input= my_data[:,1:10]
# args=(time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input)
# # Q = my_data[1:70,18]
# # print(Q)
# p0=[.01,.01,.05,1]

In [ ]:
# pars, cov = curve_fit(thermal_func,(Time,T1,T2,T3,T4,T5,T6,T7,T8,T9),T_input)

# print(pars)
# # std_devs= np.sqrt(np.diag(cov))
# # res = Q - thermal_func((T1,T2,T3,T4,T5,T6,T7,T8,T9),*pars)

# print(std_devs)
# print(res)

In [ ]:
def cooling_func(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9 = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input = args
  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9]]
  # Q=[[0],[0],[0],[0],[0],[0],[0],[0],[Q9]]
  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9]])
  k_mat = [[k12+k14, -k12, 0, -k14, 0, 0, 0, 0, 0],\
      [-k12, k12+k25+k23, -k23, 0, -k25, 0, 0, 0, 0],\
      [0, -k23, k23+k36, 0, 0, -k36, 0, 0, 0],\
      [-k14, 0, 0, k14+k45+k47, -k45, 0, -k47, 0, 0],\
      [0, -k25, 0, -k45, k25+k45+k56+k58, -k56, 0,  -k58, 0],\
      [0, 0, -k36, 0, -k56, k36+k56+k69, 0, 0, -k69],\
      [0, 0, 0, -k47, 0, 0, k47+k78, -k78, 0],\
      [0, 0, 0, 0, -k58, 0, -k78, k58+k78+k89, -k89],\
      [0,   0,   0,   0,   0,   -k69, 0, -k89, k69+k89]]


  Temp = copy.deepcopy(T)
  T_ambient = 100*np.ones((9,1))
  T_cool_arr = T
  T_temp_cool = 0
  for x in range(len(time)-1):
    i = x+1
    # T_Q9+ke9*72-(0*T1+0*T2+0*T3+0*T4+0*T5-k69*T6+0*T7-k89*T8+(k69+k89+ke9)*T9[i])*(time(i)-time(i-1))-T9(i-1)
    k_terms = ke*np.ones((9,1))*(time[i])
    convection_mat = np.exp(-k_terms)
    # print(convection_mat)
    T_cool=(-k_terms*(T_ambient - T)*convection_mat)
    T_temp_cool = T_temp_cool + T_cool*(time[i]-time[i-1])
    T_cool_arr = [T_cool_arr, T_temp_cool]
    # print(T_cool)
    print(np.matmul(k_mat,T)*(time[i]-time[i-1]))
    T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - T_cool*(time[i]-time[i-1])
    T= T - T_cool*(time[i]-time[i-1])



    Temp = np.concatenate((Temp,T),1)
  Temp_gen = Temp[8,:]
  # print(np.transpose(Temp))
  error = abs(np.transpose(Temp)-T_input)
  error=error.sum()/(len(T_input[:,1]))
  print(Temp[1,:])
  # print(error)
  # print(len(T_input[:,1]))
  return error


In [ ]:
k12 = .1
k23 = .1
k14 = .1
k25 = .1
k36 = .1
k45 = .1
k56 = .1
k47 = .1
k58 = .1
k69 = .1
k78 = .1
k89 = .1
ke1 = .01
ke2 = .01
ke3 = .01
ke4 = .01
ke5 = .01
ke6= .01
ke7 = .01
ke8 = .01
ke9 = .01
kh1 = .01
kh2 = .01
kh3 = .01
kh4 = .01
kh5 = .01
kh6 = .01
kh7 = .01
kh8 = .01
kh9 = .01

# Q9 = 0
T_reservoir = 300
init_guess = [k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_reservoir]
# temp = thermal_func((Time,T1,T2,T3,T4,T5,T6,T7,T8,T9),k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,Q9)
# print(len(temp))


In [ ]:
def unified_thermal_func(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,kh1, kh2, kh3, kh4, kh5, kh6, kh7, kh8, kh9, T_reservoir = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input = args
  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9]]
  # Q=[[0],[0],[0],[0],[0],[0],[0],[0],[Q9]]
  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9]])
  k_mat = [[k12+k14, -k12, 0, -k14, 0, 0, 0, 0, 0],\
      [-k12, k12+k25+k23, -k23, 0, -k25, 0, 0, 0, 0],\
      [0, -k23, k23+k36, 0, 0, -k36, 0, 0, 0],\
      [-k14, 0, 0, k14+k45+k47, -k45, 0, -k47, 0, 0],\
      [0, -k25, 0, -k45, k25+k45+k56+k58, -k56, 0,  -k58, 0],\
      [0, 0, -k36, 0, -k56, k36+k56+k69, 0, 0, -k69],\
      [0, 0, 0, -k47, 0, 0, k47+k78, -k78, 0],\
      [0, 0, 0, 0, -k58, 0, -k78, k58+k78+k89, -k89],\
      [0,   0,   0,   0,   0,   -k69, 0, -k89, k69+k89]]

  T_reservoir = T_reservoir*np.ones((9,1))
  Temp = copy.deepcopy(T)
  T_ambient = 100*np.ones((9,1))
  T_cool_arr = T
  T_temp_cool = 0
  for x in range(len(time)-1):
    i = x+1
    # T_Q9+ke9*72-(0*T1+0*T2+0*T3+0*T4+0*T5-k69*T6+0*T7-k89*T8+(k69+k89+ke9)*T9[i])*(time(i)-time(i-1))-T9(i-1)

    heating_node_no = 9
    heat_one_hot_vec = np.zeros((9,1))
    heat_one_hot_vec[heating_node_no-1] = 1
    k_heat_terms = k_heat*(time[i])
    heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

    k_terms = ke*np.ones((9,1))*(time[i])
    convection_mat = np.exp(-k_terms)

    # print(convection_mat)

    T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)
    #print(T_heat)

    T_cool = (-k_terms*(T - T_ambient)*convection_mat)


    T_temp_cool = T_temp_cool + T_cool*(time[i]-time[i-1])
    # print(T_cool)

    #print(np.matmul(k_mat,T)*(time[i]-time[i-1]))
    T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1])
    Temp = np.concatenate((Temp,T),1)

  Temp_gen = Temp[8,:]
  # print(np.transpose(Temp))
  error = abs(np.transpose(Temp)-T_input)
  error=error.sum()/len(T_input[:,1])
  print(Temp[8,:])
  # print(error)
  # print(len(T_input[:,1]))
  return error*5


In [ ]:
def unified_thermal_func_plot(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,kh1, kh2, kh3, kh4, kh5, kh6, kh7, kh8, kh9, T_reservoir = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T_input = args
  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9]]
  # Q=[[0],[0],[0],[0],[0],[0],[0],[0],[Q9]]
  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9]])
  k_mat = [[k12+k14, -k12, 0, -k14, 0, 0, 0, 0, 0],\
      [-k12, k12+k25+k23, -k23, 0, -k25, 0, 0, 0, 0],\
      [0, -k23, k23+k36, 0, 0, -k36, 0, 0, 0],\
      [-k14, 0, 0, k14+k45+k47, -k45, 0, -k47, 0, 0],\
      [0, -k25, 0, -k45, k25+k45+k56+k58, -k56, 0,  -k58, 0],\
      [0, 0, -k36, 0, -k56, k36+k56+k69, 0, 0, -k69],\
      [0, 0, 0, -k47, 0, 0, k47+k78, -k78, 0],\
      [0, 0, 0, 0, -k58, 0, -k78, k58+k78+k89, -k89],\
      [0,   0,   0,   0,   0,   -k69, 0, -k89, k69+k89]]

  T_reservoir = T_reservoir*np.ones((9,1))
  Temp = copy.deepcopy(T)
  T_ambient = 100*np.ones((9,1))
  T_cool_arr = T
  T_temp_cool = 0
  for x in range(len(time)-1):
    i = x+1
    # T_Q9+ke9*72-(0*T1+0*T2+0*T3+0*T4+0*T5-k69*T6+0*T7-k89*T8+(k69+k89+ke9)*T9[i])*(time(i)-time(i-1))-T9(i-1)

    heating_node_no = 9
    heat_one_hot_vec = np.zeros((9,1))
    heat_one_hot_vec[heating_node_no-1] = 1
    k_heat_terms = k_heat*(time[i])
    heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

    k_terms = ke*np.ones((9,1))*(time[i])
    convection_mat = np.exp(-k_terms)

    # print(convection_mat)

    T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)
    #print(T_heat)

    T_cool = (-k_terms*(T - T_ambient)*convection_mat)


    T_temp_cool = T_temp_cool + T_cool*(time[i]-time[i-1])
    # print(T_cool)

    #print(np.matmul(k_mat,T)*(time[i]-time[i-1]))
    T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1])
    Temp = np.concatenate((Temp,T),1)

  Temp_gen = Temp[8,:]
  # print(np.transpose(Temp))
  error = abs(np.transpose(Temp)-T_input)
  error=error.sum()/len(T_input[:,1])
  print(Temp[8,:])
  # print(error)
  # print(len(T_input[:,1]))
  return Temp

In [ ]:
my_data = genfromtxt(dataPath, delimiter=',')
my_data1 = genfromtxt(dataPath1, delimiter=',')
my_data2 = genfromtxt(dataPath2, delimiter=',')
my_data3 = genfromtxt(dataPath3, delimiter=',')
my_data4 = genfromtxt(dataPath4, delimiter=',')

test_my_data1 = genfromtxt(testdataPath1, delimiter=',')
test_my_data2 = genfromtxt(testdataPath2, delimiter=',')
test_my_data3 = genfromtxt(testdataPath3, delimiter=',')

global training_data
training_data = [[my_data1], [my_data2], [my_data3], [my_data4]]
global testing_data
testing_data = [[test_my_data1], [test_my_data2], [test_my_data3]]

# time = my_data1[:,0]
# T1=my_data1[0,1]
# T2=my_data1[0,2]
# T3=my_data1[0,3]
# T4=my_data1[0,4]
# T5=my_data1[0,5]
# T6=my_data1[0,6]
# T7=my_data1[0,7]
# T8=my_data1[0,8]
# T9=my_data1[0,9]
# T10=my_data1[0,10]
# T11=my_data1[0,11]
# T12=my_data1[0,12]
# T13=my_data1[0,13]
# T14=my_data1[0,14]
# T15=my_data1[0,15]
# T16=my_data1[0,16]
# T17=my_data1[0,17]
# T18=my_data1[0,18]
# T19=my_data1[0,19]
# T20=my_data1[0,20]
# T21=my_data1[0,21]
# heated_node_id=my_data1[0,22]


# time = my_data2[:,0]
# T1=my_data2[0,1]
# T2=my_data2[0,2]
# T3=my_data2[0,3]
# T4=my_data2[0,4]
# T5=my_data2[0,5]
# T6=my_data2[0,6]
# T7=my_data2[0,7]
# T8=my_data2[0,8]
# T9=my_data2[0,9]
# T10=my_data2[0,10]
# T11=my_data2[0,11]
# T12=my_data2[0,12]
# T13=my_data2[0,13]
# T14=my_data2[0,14]
# T15=my_data2[0,15]
# T16=my_data2[0,16]
# T17=my_data2[0,17]
# T18=my_data2[0,18]
# T19=my_data2[0,19]
# T20=my_data2[0,20]
# T21=my_data2[0,21]
# heated_node_id=my_data2[0,22]

time = my_data3[:,0]
T1=my_data3[0,1]
T2=my_data3[0,2]
T3=my_data3[0,3]
T4=my_data3[0,4]
T5=my_data3[0,5]
T6=my_data3[0,6]
T7=my_data3[0,7]
T8=my_data3[0,8]
T9=my_data3[0,9]
T10=my_data3[0,10]
T11=my_data3[0,11]
T12=my_data3[0,12]
T13=my_data3[0,13]
T14=my_data3[0,14]
T15=my_data3[0,15]
T16=my_data3[0,16]
T17=my_data3[0,17]
T18=my_data3[0,18]
T19=my_data3[0,19]
T20=my_data3[0,20]
T21=my_data3[0,21]
heated_node_id=my_data3[0,22]

# time = my_data4[:,0]
# T1=my_data4[0,1]
# T2=my_data4[0,2]
# T3=my_data4[0,3]
# T4=my_data4[0,4]
# T5=my_data4[0,5]
# T6=my_data4[0,6]
# T7=my_data4[0,7]
# T8=my_data4[0,8]
# T9=my_data4[0,9]
# T10=my_data4[0,10]
# T11=my_data4[0,11]
# T12=my_data4[0,12]
# T13=my_data4[0,13]
# T14=my_data4[0,14]
# T15=my_data4[0,15]
# T16=my_data4[0,16]
# T17=my_data4[0,17]
# T18=my_data4[0,18]
# T19=my_data4[0,19]
# T20=my_data4[0,20]
# T21=my_data4[0,21]
# heated_node_id=my_data4[0,22]

# time = test_my_data1[:,0]
# T1=test_my_data1[0,1]
# T2=test_my_data1[0,2]
# T3=test_my_data1[0,3]
# T4=test_my_data1[0,4]
# T5=test_my_data1[0,5]
# T6=test_my_data1[0,6]
# T7=test_my_data1[0,7]
# T8=test_my_data1[0,8]
# T9=test_my_data1[0,9]
# T10=test_my_data1[0,10]
# T11=test_my_data1[0,11]
# T12=test_my_data1[0,12]
# T13=test_my_data1[0,13]
# T14=test_my_data1[0,14]
# T15=test_my_data1[0,15]
# T16=test_my_data1[0,16]
# T17=test_my_data1[0,17]
# T18=test_my_data1[0,18]
# T19=test_my_data1[0,19]
# T20=test_my_data1[0,20]
# T21=test_my_data1[0,21]
# heated_node_id=test_my_data1[0,22]

# time = test_my_data2[:,0]
# T1=test_my_data2[0,1]
# T2=test_my_data2[0,2]
# T3=test_my_data2[0,3]
# T4=test_my_data2[0,4]
# T5=test_my_data2[0,5]
# T6=test_my_data2[0,6]
# T7=test_my_data2[0,7]
# T8=test_my_data2[0,8]
# T9=test_my_data2[0,9]
# T10=test_my_data2[0,10]
# T11=test_my_data2[0,11]
# T12=test_my_data2[0,12]
# T13=test_my_data2[0,13]
# T14=test_my_data2[0,14]
# T15=test_my_data2[0,15]
# T16=test_my_data2[0,16]
# T17=test_my_data2[0,17]
# T18=test_my_data2[0,18]
# T19=test_my_data2[0,19]
# T20=test_my_data2[0,20]
# T21=test_my_data2[0,21]
# heated_node_id=test_my_data2[0,22]

# time = test_my_data3[:,0]
# T1=test_my_data3[0,1]
# T2=test_my_data3[0,2]
# T3=test_my_data3[0,3]
# T4=test_my_data3[0,4]
# T5=test_my_data3[0,5]
# T6=test_my_data3[0,6]
# T7=test_my_data3[0,7]
# T8=test_my_data3[0,8]
# T9=test_my_data3[0,9]
# T10=test_my_data3[0,10]
# T11=test_my_data3[0,11]
# T12=test_my_data3[0,12]
# T13=test_my_data3[0,13]
# T14=test_my_data3[0,14]
# T15=test_my_data3[0,15]
# T16=test_my_data3[0,16]
# T17=test_my_data3[0,17]
# T18=test_my_data3[0,18]
# T19=test_my_data3[0,19]
# T20=test_my_data3[0,20]
# T21=test_my_data3[0,21]
# heated_node_id=test_my_data3[0,22]

global time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,heated_node_id
global T_input
#print(T_input)
# T_input= my_data[0:70,9]
# T_input= my_data[:,1:23]
T_input= [my_data1[:,1:23]]
# print(T_input)
args=(time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input)
# Q = my_data[1:70,18]
# print(Q)
p0=[.01,.01,.05,1]

In [ ]:
def get_neighbors(node):
  if node == 1:
    neighbors = [10,2,3,13]
    return neighbors
  elif node == 2:
    neighbors = [11,3,5,1]
    return neighbors
  elif node == 3:
    neighbors = [12,14,6,2]
    return neighbors
  elif node==4:
    neighbors = [1,5,7,15]
    return neighbors
  elif node == 5:
    neighbors = [2,6,8,4]
    return neighbors
  elif node == 6:
    neighbors = [3,16,9,5]
    return neighbors
  elif node == 7:
    neighbors = [4,8,19,17]
    return neighbors
  elif node == 8:
    neighbors = [5,9,20,7]
    return neighbors
  elif node == 9:
    neighbors = [6,18,21,8]
    return neighbors


In [ ]:
k12 = .1
k23 = .1
k14 = .1
k25 = .1
k36 = .1
k45 = .1
k56 = .1
k47 = .1
k58 = .1
k69 = .1
k78 = .1
k89 = .1
ke1 = .01
ke2 = .01
ke3 = .01
ke4 = .01
ke5 = .01
ke6= .01
ke7 = .01
ke8 = .01
ke9 = .01
ke10 = .01
ke11 = .01
ke12 = .01
ke13 = .01
ke14 = .01
ke15 = .01
ke16 = .01
ke17 = .01
ke18 = .01
ke19 = .01
ke20 = .01
ke21 = .01
kh1 = 1.0
kh2 = 1.0
kh3 = 1.0
kh4 = 1.0
kh5 = 1.0
kh6 = 1.0
kh7 = 1.0
kh8 = 1.0
kh9 = 1.0
k110 = 0.01
k113 = 0.01
k211 = 0.01
k312 = 0.01
k314 = 0.01
k415 = 0.01
k616 = 0.01
k618 = 0.01
k717 = 0.01
k719 = 0.01
k820 = 0.01
k918 = 0.01
k921 = 0.01
k1011 = 0.01
k1112 = 0.01
k1315 = 0.01
k1416 = 0.01
k1517 = 0.01
k1618 = 0.01
k1920 = 0.01
k2021 = 0.01
k15 = 0.01
k111 = 0.01
k115 = 0.01
k24 = 0.01
k26 = 0.01
k210 = 0.01
k212 = 0.01
k35 = 0.01
k311 = 0.01
k316 = 0.01
k48 = 0.01
k413 = 0.01
k417 = 0.01
k57 = 0.01
k59 = 0.01
k68 = 0.01
k614 = 0.01
k715 = 0.01
k720 = 0.01
k819 = 0.01
k821 = 0.01
k920 = 0.01
k916 = 0.01
k1013 = 0.01
k1214 = 0.01
k1719 = 0.01
k1821 = 0.01
q_k = .5
# Q9 = 0
T_reservoir = 300
T_ambient = 90
init_guess = [k12, k23, k14, k25, k36, k45, k56, k47, k58, k69, k78, k89, k110, k113, k211, k312, k314, k415, k616, k717, k719, k820, k918, k921, k1011, k1112, k1315, k1416, k1517, k1618, k1920, k2021,\
              k15,k111,k115,k24,k26,k210,k212,k35,k311,k316,k48,k413,k417,k57,k59,k68,k614,k618,k715,k720,k819,k821,k920,k916,k1013,k1214,k1719,k1821,ke1, ke2, ke3, ke4, ke5, ke6, ke7, ke8, ke9,  ke10, ke11, ke12, ke13, ke14, ke15, ke16, ke17, ke18, ke19, ke20, ke21, kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient, q_k]

In [ ]:
def unified_thermal_func_latent_node_plot(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient,T_reservoir = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input = args

  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
  # T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]

  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9],[ke10],[ke11],[ke12],[ke13],[ke14],[ke15],[ke16],[ke17],[ke18],[ke19],[ke20],[ke21]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])
  # k_mat = [[k12+k14+k113+k110, -k12, 0.0, -k14, 0.0, 0.0, 0.0, 0.0, 0.0, -k110, 0.0, 0.0, -k113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k12, k12+k25+k23+k211, -k23, 0.0, -k25, 0.0, 0.0, 0.0, 0.0, 0.0, -k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k23, k23+k36+k312+k314, 0.0, 0.0, -k36, 0.0, 0.0, 0.0, 0.0, 0.0,-k312, 0.0, -k314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k14, 0.0, 0.0, k14+k45+k47+k415, -k45, 0.0, -k47, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-k415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k25, 0.0, -k45, k25+k45+k56+k58, -k56, 0.0,  -k58, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k36, 0.0, -k56, k36+k56+k69+k616, 0.0, 0.0, -k69, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k616, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, 0.0, -k47, 0.0, 0.0, k719+k717+k47+k78, -k78, 0.0, , 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k717, 0.0, -k719, 0.0, 0.0],\
  #          [0.0, 0.0, 0.0, 0.0, -k58, 0.0, -k78, k820+k58+k78+k89, -k89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k820, 0.0],\
  #          [0.0, 0.0, 0.0, 0.0, 0.0, -k69, 0.0, -k89, k921+k918+k69+k89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k918, 0.0, 0.0, -k921],\
  #          [-k110, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k1011+k110, -k1011, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k1011, k1011+k1112+k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k1112, k1112+k312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k113+k1315, 0.0, -k1315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k314+k1416, 0.0, -k1416, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0, 0, 0, -k415, 0, 0, 0, 0, 0, 0, 0, 0, -k1315, 0, k1315+k1517+k415, 0, -k1517, 0, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, -k616, 0, 0, 0, 0, 0, 0, 0, -k1416, 0, k616+k1416+k1618, 0, -k1618, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, -k717, 0, 0, 0, 0, 0, 0, 0, -k1517, 0, k1517+k717, 0, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, 0, 0, -k918, 0, 0, 0, 0, 0, 0, -k1618, 0, k918+k1618, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, -k719, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, k719+k1920, -k1920, 0],\
  #          [0, 0, 0, 0, 0, 0, 0, -k820, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -k1920, k1920+k820+k2021, -k2021],\
  #          [0, 0, 0, 0, 0, 0, 0, 0, -k921, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -k2021, k2021+k921]]

  k_mat = np.zeros((21,21))
  k_mat[0,0] = k12+k14+k113+k110
  k_mat[0,1] = -k12
  k_mat[0,3] = -k14
  k_mat[0,9] = -k110
  k_mat[0,12] = -k113
  k_mat[1,0] = -k12
  k_mat[1,1] = k12+k25+k23+k211
  k_mat[1,2] = -k23
  k_mat[1,4] = -k25
  k_mat[1,10] = -k211
  k_mat[2,1] = -k23
  k_mat[2,2] = k23+k36+k312+k314
  k_mat[2,5] = -k36
  k_mat[2,11] = -k312
  k_mat[2,13] = -k314
  k_mat[3,0] = -k14
  k_mat[3,3] = k14+k45+k47+k415
  k_mat[3,4] = -k45
  k_mat[3,6] = -k47
  k_mat[3,14] = -k415
  k_mat[4,1] = -k25
  k_mat[4,3] = -k45
  k_mat[4,4] = k25+k45+k56+k58
  k_mat[4,5] = -k56
  k_mat[4,7] = -k58
  k_mat[5,2] = -k36
  k_mat[5,4] = -k56
  k_mat[5,5] = k36+k56+k69+k616
  k_mat[5,8] = -k69
  k_mat[5,15] = -k616
  k_mat[6,6] = k719+k717+k47+k78
  k_mat[6,3] = -k47
  k_mat[6,7] = -k78
  k_mat[6,16] = -k717
  k_mat[6,18] = -k719
  k_mat[7,7] = k820+k58+k78+k89
  k_mat[7,4] = -k58
  k_mat[7,8] = -k89
  k_mat[7,6] = -k78
  k_mat[7,19] = -k820
  k_mat[8,8] = k921+k918+k69+k89
  k_mat[8,5] = -k69
  k_mat[8,7] = -k89
  k_mat[8,17] = -k918
  k_mat[8,20] = -k921
  k_mat[9,9] = k1011+k110
  k_mat[9,0] = -k110
  k_mat[9,10] = -k1011
  k_mat[10,10] = k1011+k1112+k211
  k_mat[10,9] = -k1011
  k_mat[10,11] = -k1112
  k_mat[10,1] = -k211
  k_mat[11,11] =  k1112+k312
  k_mat[11,2] = -k312
  k_mat[11,10] = -k1112
  k_mat[12,12] = k113+k1315
  k_mat[12,2] = -k113
  k_mat[12,14] = -k1315
  k_mat[13,13] = k314+k1416
  k_mat[13,2] = -k314
  k_mat[13,15] = -k1416
  k_mat[14,14] = k1315+k1517+k415
  k_mat[14,3] = -k415
  k_mat[14,12] = -k1315
  k_mat[14,16] = -k1517
  k_mat[15,15] = k616+k1416+k1618
  k_mat[15,5] = -k616
  k_mat[15,13] = -k1416
  k_mat[15,17] = -k1618
  k_mat[16,16] = k1517+k717
  k_mat[16,14] = -k1517
  k_mat[16,6] = -k717
  k_mat[17,17] = k918+k1618
  k_mat[17,8] = -k918
  k_mat[17,15] = -k1618
  k_mat[18,18] = k719+k1920
  k_mat[18,6] = -k719
  k_mat[18,19] = -k1920
  k_mat[19,19] = k1920+k820+k2021
  k_mat[19,7] = k820
  k_mat[19,20] = -k2021
  k_mat[19,18] = -k1920
  k_mat[20,20] = k2021+k921
  k_mat[20,19] = -k2021
  k_mat[20,8] = -k921

  T_input = test_my_data1[:,1:23]

  T_reservoir = T_reservoir*np.ones((21,1))
  Temp = copy.deepcopy(T)
  T_ambient = T_ambient*np.ones((21,1))
  T_cool_arr = T
  T_temp_cool = 0

  for x in range(len(time)-1):
    i = x+1
    heating_node_no = T_input[x,-1]

    heat_one_hot_vec = np.zeros((21,1))
    if heating_node_no != 0:
      heat_one_hot_vec[int(heating_node_no)-1] = 1

    k_heat_terms = k_heat*(time[i])
    heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

    k_terms = ke*np.ones((21,1))*(time[i])
    convection_mat = np.exp(-k_terms)


    T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)  #Defining heating nodes
    T_cool = (-k_terms*(T - T_ambient)*convection_mat)    #Defining cooling nodes
    T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1]) #Predicted Temperature
    Temp = np.concatenate((Temp,T),1)

  Temp_gen = Temp[8,:]
  error = abs(np.transpose(Temp)-T_input[:,0:21])
  error=error.sum()/len(T_input[:,1])
  print(Temp[8,:])

  return Temp

In [ ]:
def unified_thermal_func_latent_node(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input):

  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient,T_reservoir = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input = args



  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9],[ke10],[ke11],[ke12],[ke13],[ke14],[ke15],[ke16],[ke17],[ke18],[ke19],[ke20],[ke21]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

  k_mat = np.zeros((21,21))
  k_mat[0,0] = k12+k14+k113+k110
  k_mat[0,1] = -k12
  k_mat[0,3] = -k14
  k_mat[0,9] = -k110
  k_mat[0,12] = -k113
  k_mat[1,0] = -k12
  k_mat[1,1] = k12+k25+k23+k211
  k_mat[1,2] = -k23
  k_mat[1,4] = -k25
  k_mat[1,10] = -k211
  k_mat[2,1] = -k23
  k_mat[2,2] = k23+k36+k312+k314
  k_mat[2,5] = -k36
  k_mat[2,11] = -k312
  k_mat[2,13] = -k314
  k_mat[3,0] = -k14
  k_mat[3,3] = k14+k45+k47+k415
  k_mat[3,4] = -k45
  k_mat[3,6] = -k47
  k_mat[3,14] = -k415
  k_mat[4,1] = -k25
  k_mat[4,3] = -k45
  k_mat[4,4] = k25+k45+k56+k58
  k_mat[4,5] = -k56
  k_mat[4,7] = -k58
  k_mat[5,2] = -k36
  k_mat[5,4] = -k56
  k_mat[5,5] = k36+k56+k69+k616
  k_mat[5,8] = -k69
  k_mat[5,15] = -k616
  k_mat[6,6] = k719+k717+k47+k78
  k_mat[6,3] = -k47
  k_mat[6,7] = -k78
  k_mat[6,16] = -k717
  k_mat[6,18] = -k719
  k_mat[7,7] = k820+k58+k78+k89
  k_mat[7,4] = -k58
  k_mat[7,8] = -k89
  k_mat[7,6] = -k78
  k_mat[7,19] = -k820
  k_mat[8,8] = k921+k918+k69+k89
  k_mat[8,5] = -k69
  k_mat[8,7] = -k89
  k_mat[8,17] = -k918
  k_mat[8,20] = -k921
  k_mat[9,9] = k1011+k110
  k_mat[9,0] = -k110
  k_mat[9,10] = -k1011
  k_mat[10,10] = k1011+k1112+k211
  k_mat[10,9] = -k1011
  k_mat[10,11] = -k1112
  k_mat[10,1] = -k211
  k_mat[11,11] =  k1112+k312
  k_mat[11,2] = -k312
  k_mat[11,10] = -k1112
  k_mat[12,12] = k113+k1315
  k_mat[12,2] = -k113
  k_mat[12,14] = -k1315
  k_mat[13,13] = k314+k1416
  k_mat[13,2] = -k314
  k_mat[13,15] = -k1416
  k_mat[14,14] = k1315+k1517+k415
  k_mat[14,3] = -k415
  k_mat[14,12] = -k1315
  k_mat[14,16] = -k1517
  k_mat[15,15] = k616+k1416+k1618
  k_mat[15,5] = -k616
  k_mat[15,13] = -k1416
  k_mat[15,17] = -k1618
  k_mat[16,16] = k1517+k717
  k_mat[16,14] = -k1517
  k_mat[16,6] = -k717
  k_mat[17,17] = k918+k1618
  k_mat[17,8] = -k918
  k_mat[17,15] = -k1618
  k_mat[18,18] = k719+k1920
  k_mat[18,6] = -k719
  k_mat[18,19] = -k1920
  k_mat[19,19] = k1920+k820+k2021
  k_mat[19,7] = k820
  k_mat[19,20] = -k2021
  k_mat[19,18] = -k1920
  k_mat[20,20] = k2021+k921
  k_mat[20,19] = -k2021
  k_mat[20,8] = -k921


  grid_size = 21
  T_reservoir = T_reservoir*np.ones((21,1))
  T_ambient = T_ambient*np.ones((21,1))

  # T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
  T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]
  error = 0
  for y in range (len(training_data)):
    my_data = np.asfarray(training_data[y])
    time = my_data[0,:,0]
    # T1=np.reshape(my_data[0,:,1], [len(my_data[0,:,1]),1])
    # T2=np.reshape(my_data[0,:,2], [len(my_data[0,:,2]),1])
    # T3=np.reshape(my_data[0,:,3], [len(my_data[0,:,3]),1])
    # T4=np.reshape(my_data[0,:,4], [len(my_data[0,:,4]),1])
    # T5=np.reshape(my_data[0,:,5], [len(my_data[0,:,5]),1])
    # T6=np.reshape(my_data[0,:,6], [len(my_data[0,:,6]),1])
    # T7=np.reshape(my_data[0,:,7], [len(my_data[0,:,7]),1])
    # T8=np.reshape(my_data[0,:,8], [len(my_data[0,:,8]),1])
    # T9=np.reshape(my_data[0,:,9], [len(my_data[0,:,9]),1])
    # T10=np.reshape(my_data[0,:,10], [len(my_data[0,:,10]),1])
    # T11=np.reshape(my_data[0,:,11], [len(my_data[0,:,11]),1])
    # T12=np.reshape(my_data[0,:,12], [len(my_data[0,:,12]),1])
    # T13=np.reshape(my_data[0,:,13], [len(my_data[0,:,13]),1])
    # T14=np.reshape(my_data[0,:,14], [len(my_data[0,:,14]),1])
    # T15=np.reshape(my_data[0,:,15], [len(my_data[0,:,15]),1])
    # T16=np.reshape(my_data[0,:,16], [len(my_data[0,:,16]),1])
    # T17=np.reshape(my_data[0,:,17], [len(my_data[0,:,17]),1])
    # T18=np.reshape(my_data[0,:,18], [len(my_data[0,:,18]),1])
    # T19=np.reshape(my_data[0,:,19], [len(my_data[0,:,19]),1])
    # T20=np.reshape(my_data[0,:,20], [len(my_data[0,:,20]),1])
    # T21=np.reshape(my_data[0,:,21], [len(my_data[0,:,21]),1])
    T1=my_data[0,0,1]
    T2=my_data[0,0,2]
    T3=my_data[0,0,3]
    T4=my_data[0,0,4]
    T5=my_data[0,0,5]
    T6=my_data[0,0,6]
    T7=my_data[0,0,7]
    T8=my_data[0,0,8]
    T9=my_data[0,0,9]
    T10=my_data[0,0,10]
    T11=my_data[0,0,11]
    T12=my_data[0,0,12]
    T13=my_data[0,0,13]
    T14=my_data[0,0,14]
    T15=my_data[0,0,15]
    T16=my_data[0,0,16]
    T17=my_data[0,0,17]
    T18=my_data[0,0,18]
    T19=my_data[0,0,19]
    T20=my_data[0,0,20]
    T21=my_data[0,0,21]

    T_input = my_data[0,:,1:23]

    T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
    # T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]
    Temp = copy.deepcopy(T)
    for x in range(len(time)-1):
      i = x+1
      heating_node_no = T_input[x,-1]
      heat_one_hot_vec = np.zeros((21,1))
      if heating_node_no != 0:
        heat_one_hot_vec[int(heating_node_no)-1] = 1

      k_heat_terms = k_heat*(time[i])
      heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

      k_terms = ke*np.ones((21,1))*(time[i])
      convection_mat = np.exp(-k_terms)


      T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)  #Defining heating nodes
      T_cool = (-k_terms*(T - T_ambient)*convection_mat)    #Defining cooling nodes
      T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1]) #Predicted Temperature
      # print(np.shape(T[0]))
      Temp = np.concatenate((Temp,T),1)
    # print(Temp[8,:])
    error_mat = abs(np.array(abs(np.transpose(Temp)-T_input[:,0:21])))

    error_mat = np.square(error_mat[0:9])
    error+=error_mat.mean()
    # error += error_mat.sum()/(len(T_input[:,1]))


  # error = error/len(training_data)
  return error

In [ ]:
def unified_thermal_diagonal_nodes(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input):

  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,k15,k111,k115,k24,k26,k210,k212,k35,k311,k316,k48,k413,k417,k57,k59,k68,k614,k618,k715,k720,k819,k821,k920,k916,k1013,k1214,k1719,k1821,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input = args



  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9],[ke10],[ke11],[ke12],[ke13],[ke14],[ke15],[ke16],[ke17],[ke18],[ke19],[ke20],[ke21]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

  k_mat = np.zeros((21,21))
  k_mat[0,0] = k12+k14+k113+k110+k111+k115+k15
  k_mat[0,1] = -k12
  k_mat[0,3] = -k14
  k_mat[0,4] = -k15
  k_mat[0,9] = -k110
  k_mat[0,10] = -k111
  k_mat[0,12] = -k113
  k_mat[0,14] = -k115
  k_mat[1,0] = -k12
  k_mat[1,1] = k12+k25+k23+k211+k210+k26+k24+k212
  k_mat[1,2] = -k23
  k_mat[1,3] = -k24
  k_mat[1,4] = -k25
  k_mat[1,5] = -k26
  k_mat[1,9] = -k210
  k_mat[1,10] = -k211
  k_mat[1,11] = -k212
  k_mat[2,1] = -k23
  k_mat[2,2] = k23+k36+k312+k314+k311+k316+k35
  k_mat[2,4] = -k35
  k_mat[2,5] = -k36
  k_mat[2,10] = -k311
  k_mat[2,11] = -k312
  k_mat[2,13] = -k314
  k_mat[2,15] = -k316
  k_mat[3,0] = -k14
  k_mat[3,3] = k14+k45+k47+k415+k413+k417+k48+k24
  k_mat[3,1] = -k24
  k_mat[3,7] = -k48
  k_mat[3,4] = -k45
  k_mat[3,6] = -k47
  k_mat[3,12] = -k413
  k_mat[3,14] = -k415
  k_mat[3,16] = -k417
  k_mat[4,0] = -k15
  k_mat[4,1] = -k25
  k_mat[4,2] = -k35
  k_mat[4,3] = -k45
  k_mat[4,4] = k25+k45+k56+k58+k15+k57+k59+k35
  k_mat[4,5] = -k56
  k_mat[4,6] = -k57
  k_mat[4,7] = -k58
  k_mat[4,8] = -k59
  k_mat[5,1] = -k26
  k_mat[5,2] = -k36
  k_mat[5,4] = -k56
  k_mat[5,5] = k36+k56+k69+k616+k26+k614+k618+k68
  k_mat[5,7] = -k68
  k_mat[5,13] = -k614
  k_mat[5,8] = -k69
  k_mat[5,17] = -k618
  k_mat[5,15] = -k616
  k_mat[6,6] = k719+k717+k47+k78+k715+k57+k720
  k_mat[6,3] = -k47
  k_mat[6,4] = -k57
  k_mat[6,7] = -k78
  k_mat[6,14] = -k715
  k_mat[6,16] = -k717
  k_mat[6,18] = -k719
  k_mat[6,19] = -k720
  k_mat[7,7] = k820+k58+k78+k89+k48+k68+k819+k821
  k_mat[7,3] = -k48
  k_mat[7,4] = -k58
  k_mat[7,8] = -k89
  k_mat[7,6] = -k78
  k_mat[7,5] = -k68
  k_mat[7,18] = -k819
  k_mat[7,19] = -k820
  k_mat[7,20] = -k821
  k_mat[8,8] = k921+k918+k69+k89+k59+k916+k920
  k_mat[8,4] = -k59
  k_mat[8,5] = -k69
  k_mat[8,7] = -k89
  k_mat[8,15] = -k916
  k_mat[8,17] = -k918
  k_mat[8,19] = -k920
  k_mat[8,20] = -k921
  k_mat[9,9] = k1011+k110+k1013+k210
  k_mat[9,0] = -k110
  k_mat[9,1] = -k210
  k_mat[9,10] = -k1011
  k_mat[9,12] = -k1013
  k_mat[10,10] = k1011+k1112+k211+k111+k311
  k_mat[10,0] = -k111
  k_mat[10,2] = -k311
  k_mat[10,9] = -k1011
  k_mat[10,11] = -k1112
  k_mat[10,1] = -k211
  k_mat[11,11] =  k1112+k312+k212+k1214
  k_mat[11,1] = -k212
  k_mat[11,2] = -k312
  k_mat[11,10] = -k1112
  k_mat[11,13] = -k1214
  k_mat[12,12] = k113+k1315+k1013+k413
  k_mat[12,3] = -k413
  k_mat[12,0] = -k113
  k_mat[12,9] = -k1013
  k_mat[12,14] = -k1315
  k_mat[13,13] = k314+k1416+k1214+k614
  k_mat[13,5] = -k614
  k_mat[13,2] = -k314
  k_mat[13,11] = -k1214
  k_mat[13,15] = -k1416
  k_mat[14,14] = k1315+k1517+k415+k115+k715
  k_mat[14,0] = -k115
  k_mat[14,3] = -k415
  k_mat[14,6] = -k715
  k_mat[14,12] = -k1315
  k_mat[14,16] = -k1517
  k_mat[15,15] = k616+k1416+k1618+k316+k916
  k_mat[15,2] = -k316
  k_mat[15,5] = -k616
  k_mat[15,8] = -k916
  k_mat[15,13] = -k1416
  k_mat[15,17] = -k1618
  k_mat[16,16] = k1517+k717+k417+k1719
  k_mat[16,3] = -k417
  k_mat[16,14] = -k1517
  k_mat[16,18] = -k1719
  k_mat[16,6] = -k717
  k_mat[17,17] = k918+k1618+k618+k1821
  k_mat[17,8] = -k918
  k_mat[17,15] = -k1618
  k_mat[17,20] = -k1821
  k_mat[18,18] = k719+k1920+k1719+k819
  k_mat[18,7] = -k819
  k_mat[18,6] = -k719
  k_mat[18,16] = -k1719
  k_mat[18,19] = -k1920
  k_mat[19,19] = k1920+k820+k2021+k720+k920
  k_mat[19,7] = -k820
  k_mat[19,6] = -k720
  k_mat[19,8] = -k920
  k_mat[19,20] = -k2021
  k_mat[19,18] = -k1920
  k_mat[20,20] = k2021+k921+k1821+k821
  k_mat[20,7] = -k821
  k_mat[20,17] = -k1821
  k_mat[20,19] = -k2021
  k_mat[20,8] = -k921


  grid_size = 21
  # T_reservoir = T_reservoir*np.ones((21,1))
  T_ambient = T_ambient*np.ones((21,1))

  # T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
  T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]
  error = 0
  for y in range (len(training_data)):
    my_data = np.asfarray(training_data[y])
    time = my_data[0,:,0]

    T1=my_data[0,0,1]
    T2=my_data[0,0,2]
    T3=my_data[0,0,3]
    T4=my_data[0,0,4]
    T5=my_data[0,0,5]
    T6=my_data[0,0,6]
    T7=my_data[0,0,7]
    T8=my_data[0,0,8]
    T9=my_data[0,0,9]
    T10=my_data[0,0,10]
    T11=my_data[0,0,11]
    T12=my_data[0,0,12]
    T13=my_data[0,0,13]
    T14=my_data[0,0,14]
    T15=my_data[0,0,15]
    T16=my_data[0,0,16]
    T17=my_data[0,0,17]
    T18=my_data[0,0,18]
    T19=my_data[0,0,19]
    T20=my_data[0,0,20]
    T21=my_data[0,0,21]

    T_input = my_data[0,:,1:23]

    T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
    # T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]
    Temp = copy.deepcopy(T)
    for x in range(len(time)-1):
      i = x+1
      heating_node_no = T_input[x,-1]
      heat_one_hot_vec = np.zeros((21,1))
      if heating_node_no != 0:
        heat_one_hot_vec[int(heating_node_no)-1] = 1

      # k_heat_terms = k_heat*(time[i])
      # heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

      # k_terms = ke*np.ones((21,1))*(time[i])
      # convection_mat = np.exp(-k_terms)


      # T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)  #Defining heating nodes
      # T_cool = (-k_terms*(T - T_ambient)*convection_mat)    #Defining cooling nodes
      # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1]) #Predicted Temperature
      # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) - ((k_heat*(T - T_reservoir))*heat_one_hot_vec*(time[i]-time[i-1]))
      T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) + k_heat*heat_one_hot_vec
      # print(np.shape(T[0]))
      Temp = np.concatenate((Temp,T),1)
    # print(Temp[8,:])
    error_mat = np.array(abs(np.transpose(Temp)-T_input[:,0:21]))
    error_mat = np.square(error_mat)
    error_mat[0:9]=error_mat[0:9]*10
    error+=np.sum(np.sum(error_mat))
    # error += error_mat.sum()/(len(T_input[:,1]))

    error=error/10000
  # error = error/len(training_data)
  return error

In [ ]:
def unified_thermal_diagonal_node_plot(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,k15,k111,k115,k24,k26,k210,k212,k35,k311,k316,k48,k413,k417,k57,k59,k68,k614,k618,k715,k720,k819,k821,k920,k916,k1013,k1214,k1719,k1821,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input = args


  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
  # T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]

  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9],[ke10],[ke11],[ke12],[ke13],[ke14],[ke15],[ke16],[ke17],[ke18],[ke19],[ke20],[ke21]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])
  # k_mat = [[k12+k14+k113+k110, -k12, 0.0, -k14, 0.0, 0.0, 0.0, 0.0, 0.0, -k110, 0.0, 0.0, -k113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k12, k12+k25+k23+k211, -k23, 0.0, -k25, 0.0, 0.0, 0.0, 0.0, 0.0, -k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k23, k23+k36+k312+k314, 0.0, 0.0, -k36, 0.0, 0.0, 0.0, 0.0, 0.0,-k312, 0.0, -k314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k14, 0.0, 0.0, k14+k45+k47+k415, -k45, 0.0, -k47, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-k415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k25, 0.0, -k45, k25+k45+k56+k58, -k56, 0.0,  -k58, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k36, 0.0, -k56, k36+k56+k69+k616, 0.0, 0.0, -k69, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k616, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, 0.0, -k47, 0.0, 0.0, k719+k717+k47+k78, -k78, 0.0, , 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k717, 0.0, -k719, 0.0, 0.0],\
  #          [0.0, 0.0, 0.0, 0.0, -k58, 0.0, -k78, k820+k58+k78+k89, -k89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k820, 0.0],\
  #          [0.0, 0.0, 0.0, 0.0, 0.0, -k69, 0.0, -k89, k921+k918+k69+k89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k918, 0.0, 0.0, -k921],\
  #          [-k110, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k1011+k110, -k1011, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k1011, k1011+k1112+k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k1112, k1112+k312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k113+k1315, 0.0, -k1315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k314+k1416, 0.0, -k1416, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0, 0, 0, -k415, 0, 0, 0, 0, 0, 0, 0, 0, -k1315, 0, k1315+k1517+k415, 0, -k1517, 0, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, -k616, 0, 0, 0, 0, 0, 0, 0, -k1416, 0, k616+k1416+k1618, 0, -k1618, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, -k717, 0, 0, 0, 0, 0, 0, 0, -k1517, 0, k1517+k717, 0, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, 0, 0, -k918, 0, 0, 0, 0, 0, 0, -k1618, 0, k918+k1618, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, -k719, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, k719+k1920, -k1920, 0],\
  #          [0, 0, 0, 0, 0, 0, 0, -k820, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -k1920, k1920+k820+k2021, -k2021],\
  #          [0, 0, 0, 0, 0, 0, 0, 0, -k921, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -k2021, k2021+k921]]

  k_mat = np.zeros((21,21))
  k_mat[0,0] = k12+k14+k113+k110+k111+k115+k15
  k_mat[0,1] = -k12
  k_mat[0,3] = -k14
  k_mat[0,4] = -k15
  k_mat[0,9] = -k110
  k_mat[0,10] = -k111
  k_mat[0,12] = -k113
  k_mat[0,14] = -k115
  k_mat[1,0] = -k12
  k_mat[1,1] = k12+k25+k23+k211+k210+k26+k24+k212
  k_mat[1,2] = -k23
  k_mat[1,3] = -k24
  k_mat[1,4] = -k25
  k_mat[1,5] = -k26
  k_mat[1,9] = -k210
  k_mat[1,10] = -k211
  k_mat[1,11] = -k212
  k_mat[2,1] = -k23
  k_mat[2,2] = k23+k36+k312+k314+k311+k316+k35
  k_mat[2,4] = -k35
  k_mat[2,5] = -k36
  k_mat[2,10] = -k311
  k_mat[2,11] = -k312
  k_mat[2,13] = -k314
  k_mat[2,15] = -k316
  k_mat[3,0] = -k14
  k_mat[3,3] = k14+k45+k47+k415+k413+k417+k48+k24
  k_mat[3,1] = -k24
  k_mat[3,7] = -k48
  k_mat[3,4] = -k45
  k_mat[3,6] = -k47
  k_mat[3,12] = -k413
  k_mat[3,14] = -k415
  k_mat[3,16] = -k417
  k_mat[4,0] = -k15
  k_mat[4,1] = -k25
  k_mat[4,2] = -k35
  k_mat[4,3] = -k45
  k_mat[4,4] = k25+k45+k56+k58+k15+k57+k59+k35
  k_mat[4,5] = -k56
  k_mat[4,6] = -k57
  k_mat[4,7] = -k58
  k_mat[4,8] = -k59
  k_mat[5,1] = -k26
  k_mat[5,2] = -k36
  k_mat[5,4] = -k56
  k_mat[5,5] = k36+k56+k69+k616+k26+k614+k618+k68
  k_mat[5,7] = -k68
  k_mat[5,13] = -k614
  k_mat[5,8] = -k69
  k_mat[5,17] = -k618
  k_mat[5,15] = -k616
  k_mat[6,6] = k719+k717+k47+k78+k715+k57+k720
  k_mat[6,3] = -k47
  k_mat[6,4] = -k57
  k_mat[6,7] = -k78
  k_mat[6,14] = -k715
  k_mat[6,16] = -k717
  k_mat[6,18] = -k719
  k_mat[6,19] = -k720
  k_mat[7,7] = k820+k58+k78+k89+k48+k68+k819+k821
  k_mat[7,3] = -k48
  k_mat[7,4] = -k58
  k_mat[7,8] = -k89
  k_mat[7,6] = -k78
  k_mat[7,5] = -k68
  k_mat[7,18] = -k819
  k_mat[7,19] = -k820
  k_mat[7,20] = -k821
  k_mat[8,8] = k921+k918+k69+k89+k59+k916+k920
  k_mat[8,4] = -k59
  k_mat[8,5] = -k69
  k_mat[8,7] = -k89
  k_mat[8,15] = -k916
  k_mat[8,17] = -k918
  k_mat[8,19] = -k920
  k_mat[8,20] = -k921
  k_mat[9,9] = k1011+k110+k1013+k210
  k_mat[9,0] = -k110
  k_mat[9,1] = -k210
  k_mat[9,10] = -k1011
  k_mat[9,12] = -k1013
  k_mat[10,10] = k1011+k1112+k211+k111+k311
  k_mat[10,0] = -k111
  k_mat[10,2] = -k311
  k_mat[10,9] = -k1011
  k_mat[10,11] = -k1112
  k_mat[10,1] = -k211
  k_mat[11,11] =  k1112+k312+k212+k1214
  k_mat[11,1] = -k212
  k_mat[11,2] = -k312
  k_mat[11,10] = -k1112
  k_mat[11,13] = -k1214
  k_mat[12,12] = k113+k1315+k1013+k413
  k_mat[12,3] = -k413
  k_mat[12,0] = -k113
  k_mat[12,9] = -k1013
  k_mat[12,14] = -k1315
  k_mat[13,13] = k314+k1416+k1214+k614
  k_mat[13,5] = -k614
  k_mat[13,2] = -k314
  k_mat[13,11] = -k1214
  k_mat[13,15] = -k1416
  k_mat[14,14] = k1315+k1517+k415+k115+k715
  k_mat[14,0] = -k115
  k_mat[14,3] = -k415
  k_mat[14,6] = -k715
  k_mat[14,12] = -k1315
  k_mat[14,16] = -k1517
  k_mat[15,15] = k616+k1416+k1618+k316+k916
  k_mat[15,2] = -k316
  k_mat[15,5] = -k616
  k_mat[15,8] = -k916
  k_mat[15,13] = -k1416
  k_mat[15,17] = -k1618
  k_mat[16,16] = k1517+k717+k417+k1719
  k_mat[16,3] = -k417
  k_mat[16,14] = -k1517
  k_mat[16,18] = -k1719
  k_mat[16,6] = -k717
  k_mat[17,17] = k918+k1618+k618+k1821
  k_mat[17,8] = -k918
  k_mat[17,15] = -k1618
  k_mat[17,20] = -k1821
  k_mat[18,18] = k719+k1920+k1719+k819
  k_mat[18,7] = -k819
  k_mat[18,6] = -k719
  k_mat[18,16] = -k1719
  k_mat[18,19] = -k1920
  k_mat[19,19] = k1920+k820+k2021+k720+k920
  k_mat[19,7] = -k820
  k_mat[19,6] = -k720
  k_mat[19,8] = -k920
  k_mat[19,20] = -k2021
  k_mat[19,18] = -k1920
  k_mat[20,20] = k2021+k921+k1821+k821
  k_mat[20,7] = -k821
  k_mat[20,17] = -k1821
  k_mat[20,19] = -k2021
  k_mat[20,8] = -k921


  T_input = my_data3[:,1:23]

  # T_reservoir = T_reservoir*np.ones((21,1))
  Temp = copy.deepcopy(T)
  T_ambient = T_ambient*np.ones((21,1))
  T_cool_arr = T
  T_temp_cool = 0
  error=0

  for x in range(len(time)-1):
    i = x+1
    heating_node_no = T_input[x,-1]
    heat_one_hot_vec = np.zeros((21,1))
    if heating_node_no != 0:
      heat_one_hot_vec[int(heating_node_no)-1] = 1

    # k_heat_terms = k_heat*(time[i])
    # heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

    # k_terms = ke*np.ones((21,1))*(time[i])
    # convection_mat = np.exp(-k_terms)


    # T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)  #Defining heating nodes
    # T_cool = (-k_terms*(T - T_ambient)*convection_mat)    #Defining cooling nodes
    # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1]) #Predicted Temperature
    # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) - ((k_heat*(T - T_reservoir))*heat_one_hot_vec*(time[i]-time[i-1]))
    T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) + k_heat*heat_one_hot_vec
    # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) + k_heat
    ke######################DONT FORGET TO READD K
    # print(np.shape(T[0]))
    Temp = np.concatenate((Temp,T),1)
    # print(Temp[8,:])
  # error_mat = np.array(abs(np.transpose(Temp)-T_input[:,0:21]))
  # error_mat = np.square(error_mat)
  # error+=error_mat.mean()
  print(Temp[8,:])

  return Temp

In [ ]:
def unified_thermal_multi_q(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input):

  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,k15,k111,k115,k24,k26,k210,k212,k35,k311,k316,k48,k413,k417,k57,k59,k68,k614,k618,k715,k720,k819,k821,k920,k916,k1013,k1214,k1719,k1821,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient, q_k = input
  time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input = args



  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9],[ke10],[ke11],[ke12],[ke13],[ke14],[ke15],[ke16],[ke17],[ke18],[ke19],[ke20],[ke21]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

  k_mat = np.zeros((21,21))
  k_mat[0,0] = k12+k14+k113+k110+k111+k115+k15
  k_mat[0,1] = -k12
  k_mat[0,3] = -k14
  k_mat[0,4] = -k15
  k_mat[0,9] = -k110
  k_mat[0,10] = -k111
  k_mat[0,12] = -k113
  k_mat[0,14] = -k115
  k_mat[1,0] = -k12
  k_mat[1,1] = k12+k25+k23+k211+k210+k26+k24+k212
  k_mat[1,2] = -k23
  k_mat[1,3] = -k24
  k_mat[1,4] = -k25
  k_mat[1,5] = -k26
  k_mat[1,9] = -k210
  k_mat[1,10] = -k211
  k_mat[1,11] = -k212
  k_mat[2,1] = -k23
  k_mat[2,2] = k23+k36+k312+k314+k311+k316+k35
  k_mat[2,4] = -k35
  k_mat[2,5] = -k36
  k_mat[2,10] = -k311
  k_mat[2,11] = -k312
  k_mat[2,13] = -k314
  k_mat[2,15] = -k316
  k_mat[3,0] = -k14
  k_mat[3,3] = k14+k45+k47+k415+k413+k417+k48+k24
  k_mat[3,1] = -k24
  k_mat[3,7] = -k48
  k_mat[3,4] = -k45
  k_mat[3,6] = -k47
  k_mat[3,12] = -k413
  k_mat[3,14] = -k415
  k_mat[3,16] = -k417
  k_mat[4,0] = -k15
  k_mat[4,1] = -k25
  k_mat[4,2] = -k35
  k_mat[4,3] = -k45
  k_mat[4,4] = k25+k45+k56+k58+k15+k57+k59+k35
  k_mat[4,5] = -k56
  k_mat[4,6] = -k57
  k_mat[4,7] = -k58
  k_mat[4,8] = -k59
  k_mat[5,1] = -k26
  k_mat[5,2] = -k36
  k_mat[5,4] = -k56
  k_mat[5,5] = k36+k56+k69+k616+k26+k614+k618+k68
  k_mat[5,7] = -k68
  k_mat[5,13] = -k614
  k_mat[5,8] = -k69
  k_mat[5,17] = -k618
  k_mat[5,15] = -k616
  k_mat[6,6] = k719+k717+k47+k78+k715+k57+k720
  k_mat[6,3] = -k47
  k_mat[6,4] = -k57
  k_mat[6,7] = -k78
  k_mat[6,14] = -k715
  k_mat[6,16] = -k717
  k_mat[6,18] = -k719
  k_mat[6,19] = -k720
  k_mat[7,7] = k820+k58+k78+k89+k48+k68+k819+k821
  k_mat[7,3] = -k48
  k_mat[7,4] = -k58
  k_mat[7,8] = -k89
  k_mat[7,6] = -k78
  k_mat[7,5] = -k68
  k_mat[7,18] = -k819
  k_mat[7,19] = -k820
  k_mat[7,20] = -k821
  k_mat[8,8] = k921+k918+k69+k89+k59+k916+k920
  k_mat[8,4] = -k59
  k_mat[8,5] = -k69
  k_mat[8,7] = -k89
  k_mat[8,15] = -k916
  k_mat[8,17] = -k918
  k_mat[8,19] = -k920
  k_mat[8,20] = -k921
  k_mat[9,9] = k1011+k110+k1013+k210
  k_mat[9,0] = -k110
  k_mat[9,1] = -k210
  k_mat[9,10] = -k1011
  k_mat[9,12] = -k1013
  k_mat[10,10] = k1011+k1112+k211+k111+k311
  k_mat[10,0] = -k111
  k_mat[10,2] = -k311
  k_mat[10,9] = -k1011
  k_mat[10,11] = -k1112
  k_mat[10,1] = -k211
  k_mat[11,11] =  k1112+k312+k212+k1214
  k_mat[11,1] = -k212
  k_mat[11,2] = -k312
  k_mat[11,10] = -k1112
  k_mat[11,13] = -k1214
  k_mat[12,12] = k113+k1315+k1013+k413
  k_mat[12,3] = -k413
  k_mat[12,0] = -k113
  k_mat[12,9] = -k1013
  k_mat[12,14] = -k1315
  k_mat[13,13] = k314+k1416+k1214+k614
  k_mat[13,5] = -k614
  k_mat[13,2] = -k314
  k_mat[13,11] = -k1214
  k_mat[13,15] = -k1416
  k_mat[14,14] = k1315+k1517+k415+k115+k715
  k_mat[14,0] = -k115
  k_mat[14,3] = -k415
  k_mat[14,6] = -k715
  k_mat[14,12] = -k1315
  k_mat[14,16] = -k1517
  k_mat[15,15] = k616+k1416+k1618+k316+k916
  k_mat[15,2] = -k316
  k_mat[15,5] = -k616
  k_mat[15,8] = -k916
  k_mat[15,13] = -k1416
  k_mat[15,17] = -k1618
  k_mat[16,16] = k1517+k717+k417+k1719
  k_mat[16,3] = -k417
  k_mat[16,14] = -k1517
  k_mat[16,18] = -k1719
  k_mat[16,6] = -k717
  k_mat[17,17] = k918+k1618+k618+k1821
  k_mat[17,8] = -k918
  k_mat[17,15] = -k1618
  k_mat[17,20] = -k1821
  k_mat[18,18] = k719+k1920+k1719+k819
  k_mat[18,7] = -k819
  k_mat[18,6] = -k719
  k_mat[18,16] = -k1719
  k_mat[18,19] = -k1920
  k_mat[19,19] = k1920+k820+k2021+k720+k920
  k_mat[19,7] = -k820
  k_mat[19,6] = -k720
  k_mat[19,8] = -k920
  k_mat[19,20] = -k2021
  k_mat[19,18] = -k1920
  k_mat[20,20] = k2021+k921+k1821+k821
  k_mat[20,7] = -k821
  k_mat[20,17] = -k1821
  k_mat[20,19] = -k2021
  k_mat[20,8] = -k921


  grid_size = 21
  # T_reservoir = T_reservoir*np.ones((21,1))
  T_ambient = T_ambient*np.ones((21,1))

  # T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
  T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]
  error = 0
  for y in range (len(training_data)):
    my_data = np.asfarray(training_data[y])
    time = my_data[0,:,0]

    T1=my_data[0,0,1]
    T2=my_data[0,0,2]
    T3=my_data[0,0,3]
    T4=my_data[0,0,4]
    T5=my_data[0,0,5]
    T6=my_data[0,0,6]
    T7=my_data[0,0,7]
    T8=my_data[0,0,8]
    T9=my_data[0,0,9]
    T10=my_data[0,0,10]
    T11=my_data[0,0,11]
    T12=my_data[0,0,12]
    T13=my_data[0,0,13]
    T14=my_data[0,0,14]
    T15=my_data[0,0,15]
    T16=my_data[0,0,16]
    T17=my_data[0,0,17]
    T18=my_data[0,0,18]
    T19=my_data[0,0,19]
    T20=my_data[0,0,20]
    T21=my_data[0,0,21]

    T_input = my_data[0,:,1:23]

    T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
    # T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]
    Temp = copy.deepcopy(T)
    for x in range(len(time)-1):
      i = x+1
      heating_node_no = T_input[x,-1]
      heat_one_hot_vec = np.zeros((21,1))
      if heating_node_no != 0:
        heat_one_hot_vec[int(heating_node_no)-1] = 1
        neighbors = get_neighbors(int(heating_node_no))
        for neighbor in neighbors:
          heat_one_hot_vec[neighbor-1] = q_k
      # k_heat_terms = k_heat*(time[i])
      # heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

      # k_terms = ke*np.ones((21,1))*(time[i])
      # convection_mat = np.exp(-k_terms)


      # T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)  #Defining heating nodes
      # T_cool = (-k_terms*(T - T_ambient)*convection_mat)    #Defining cooling nodes
      # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1]) #Predicted Temperature
      # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) - ((k_heat*(T - T_reservoir))*heat_one_hot_vec*(time[i]-time[i-1]))
      T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) + k_heat*heat_one_hot_vec
      # print(np.shape(T[0]))
      Temp = np.concatenate((Temp,T),1)
    # print(Temp[8,:])
    error_mat = np.array(abs(np.transpose(Temp)-T_input[:,0:21]))
    error_mat = np.square(error_mat)
    error_mat[0:9]=error_mat[0:9]*10
    error+=np.sum(np.sum(error_mat))
    # error += error_mat.sum()/(len(T_input[:,1]))

    error=error/10
  # error = error/len(training_data)
  return error

In [ ]:
def unified_thermal_multi_q_plot(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input):
  k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,k15,k111,k115,k24,k26,k210,k212,k35,k311,k316,k48,k413,k417,k57,k59,k68,k614,k618,k715,k720,k819,k821,k920,k916,k1013,k1214,k1719,k1821,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient, q_k = input
  # time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input = args


  T=[[T1],[T2],[T3],[T4],[T5],[T6],[T7],[T8],[T9],[T10],[T11],[T12],[T13],[T14],[T15],[T16],[T17],[T18],[T19],[T20],[T21]]
  # T=[T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21]

  ke =np.array([[ke1],[ke2],[ke3],[ke4],[ke5],[ke6],[ke7],[ke8],[ke9],[ke10],[ke11],[ke12],[ke13],[ke14],[ke15],[ke16],[ke17],[ke18],[ke19],[ke20],[ke21]])
  k_heat = np.array([[kh1],[kh2],[kh3],[kh4],[kh5],[kh6],[kh7],[kh8],[kh9], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])
  # k_mat = [[k12+k14+k113+k110, -k12, 0.0, -k14, 0.0, 0.0, 0.0, 0.0, 0.0, -k110, 0.0, 0.0, -k113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k12, k12+k25+k23+k211, -k23, 0.0, -k25, 0.0, 0.0, 0.0, 0.0, 0.0, -k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k23, k23+k36+k312+k314, 0.0, 0.0, -k36, 0.0, 0.0, 0.0, 0.0, 0.0,-k312, 0.0, -k314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k14, 0.0, 0.0, k14+k45+k47+k415, -k45, 0.0, -k47, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,-k415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k25, 0.0, -k45, k25+k45+k56+k58, -k56, 0.0,  -k58, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k36, 0.0, -k56, k36+k56+k69+k616, 0.0, 0.0, -k69, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k616, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, 0.0, -k47, 0.0, 0.0, k719+k717+k47+k78, -k78, 0.0, , 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k717, 0.0, -k719, 0.0, 0.0],\
  #          [0.0, 0.0, 0.0, 0.0, -k58, 0.0, -k78, k820+k58+k78+k89, -k89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k820, 0.0],\
  #          [0.0, 0.0, 0.0, 0.0, 0.0, -k69, 0.0, -k89, k921+k918+k69+k89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k918, 0.0, 0.0, -k921],\
  #          [-k110, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k1011+k110, -k1011, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, -k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k1011, k1011+k1112+k211, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -k1112, k1112+k312, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [-k113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k113+k1315, 0.0, -k1315, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0.0, 0.0, -k314, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, k314+k1416, 0.0, -k1416, 0.0, 0.0, 0.0, 0.0, 0.0],\
  #          [0, 0, 0, -k415, 0, 0, 0, 0, 0, 0, 0, 0, -k1315, 0, k1315+k1517+k415, 0, -k1517, 0, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, -k616, 0, 0, 0, 0, 0, 0, 0, -k1416, 0, k616+k1416+k1618, 0, -k1618, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, -k717, 0, 0, 0, 0, 0, 0, 0, -k1517, 0, k1517+k717, 0, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, 0, 0, -k918, 0, 0, 0, 0, 0, 0, -k1618, 0, k918+k1618, 0, 0, 0],\
  #          [0, 0, 0, 0, 0, 0, -k719, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, k719+k1920, -k1920, 0],\
  #          [0, 0, 0, 0, 0, 0, 0, -k820, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -k1920, k1920+k820+k2021, -k2021],\
  #          [0, 0, 0, 0, 0, 0, 0, 0, -k921, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -k2021, k2021+k921]]

  k_mat = np.zeros((21,21))
  k_mat[0,0] = k12+k14+k113+k110+k111+k115+k15
  k_mat[0,1] = -k12
  k_mat[0,3] = -k14
  k_mat[0,4] = -k15
  k_mat[0,9] = -k110
  k_mat[0,10] = -k111
  k_mat[0,12] = -k113
  k_mat[0,14] = -k115
  k_mat[1,0] = -k12
  k_mat[1,1] = k12+k25+k23+k211+k210+k26+k24+k212
  k_mat[1,2] = -k23
  k_mat[1,3] = -k24
  k_mat[1,4] = -k25
  k_mat[1,5] = -k26
  k_mat[1,9] = -k210
  k_mat[1,10] = -k211
  k_mat[1,11] = -k212
  k_mat[2,1] = -k23
  k_mat[2,2] = k23+k36+k312+k314+k311+k316+k35
  k_mat[2,4] = -k35
  k_mat[2,5] = -k36
  k_mat[2,10] = -k311
  k_mat[2,11] = -k312
  k_mat[2,13] = -k314
  k_mat[2,15] = -k316
  k_mat[3,0] = -k14
  k_mat[3,3] = k14+k45+k47+k415+k413+k417+k48+k24
  k_mat[3,1] = -k24
  k_mat[3,7] = -k48
  k_mat[3,4] = -k45
  k_mat[3,6] = -k47
  k_mat[3,12] = -k413
  k_mat[3,14] = -k415
  k_mat[3,16] = -k417
  k_mat[4,0] = -k15
  k_mat[4,1] = -k25
  k_mat[4,2] = -k35
  k_mat[4,3] = -k45
  k_mat[4,4] = k25+k45+k56+k58+k15+k57+k59+k35
  k_mat[4,5] = -k56
  k_mat[4,6] = -k57
  k_mat[4,7] = -k58
  k_mat[4,8] = -k59
  k_mat[5,1] = -k26
  k_mat[5,2] = -k36
  k_mat[5,4] = -k56
  k_mat[5,5] = k36+k56+k69+k616+k26+k614+k618+k68
  k_mat[5,7] = -k68
  k_mat[5,13] = -k614
  k_mat[5,8] = -k69
  k_mat[5,17] = -k618
  k_mat[5,15] = -k616
  k_mat[6,6] = k719+k717+k47+k78+k715+k57+k720
  k_mat[6,3] = -k47
  k_mat[6,4] = -k57
  k_mat[6,7] = -k78
  k_mat[6,14] = -k715
  k_mat[6,16] = -k717
  k_mat[6,18] = -k719
  k_mat[6,19] = -k720
  k_mat[7,7] = k820+k58+k78+k89+k48+k68+k819+k821
  k_mat[7,3] = -k48
  k_mat[7,4] = -k58
  k_mat[7,8] = -k89
  k_mat[7,6] = -k78
  k_mat[7,5] = -k68
  k_mat[7,18] = -k819
  k_mat[7,19] = -k820
  k_mat[7,20] = -k821
  k_mat[8,8] = k921+k918+k69+k89+k59+k916+k920
  k_mat[8,4] = -k59
  k_mat[8,5] = -k69
  k_mat[8,7] = -k89
  k_mat[8,15] = -k916
  k_mat[8,17] = -k918
  k_mat[8,19] = -k920
  k_mat[8,20] = -k921
  k_mat[9,9] = k1011+k110+k1013+k210
  k_mat[9,0] = -k110
  k_mat[9,1] = -k210
  k_mat[9,10] = -k1011
  k_mat[9,12] = -k1013
  k_mat[10,10] = k1011+k1112+k211+k111+k311
  k_mat[10,0] = -k111
  k_mat[10,2] = -k311
  k_mat[10,9] = -k1011
  k_mat[10,11] = -k1112
  k_mat[10,1] = -k211
  k_mat[11,11] =  k1112+k312+k212+k1214
  k_mat[11,1] = -k212
  k_mat[11,2] = -k312
  k_mat[11,10] = -k1112
  k_mat[11,13] = -k1214
  k_mat[12,12] = k113+k1315+k1013+k413
  k_mat[12,3] = -k413
  k_mat[12,0] = -k113
  k_mat[12,9] = -k1013
  k_mat[12,14] = -k1315
  k_mat[13,13] = k314+k1416+k1214+k614
  k_mat[13,5] = -k614
  k_mat[13,2] = -k314
  k_mat[13,11] = -k1214
  k_mat[13,15] = -k1416
  k_mat[14,14] = k1315+k1517+k415+k115+k715
  k_mat[14,0] = -k115
  k_mat[14,3] = -k415
  k_mat[14,6] = -k715
  k_mat[14,12] = -k1315
  k_mat[14,16] = -k1517
  k_mat[15,15] = k616+k1416+k1618+k316+k916
  k_mat[15,2] = -k316
  k_mat[15,5] = -k616
  k_mat[15,8] = -k916
  k_mat[15,13] = -k1416
  k_mat[15,17] = -k1618
  k_mat[16,16] = k1517+k717+k417+k1719
  k_mat[16,3] = -k417
  k_mat[16,14] = -k1517
  k_mat[16,18] = -k1719
  k_mat[16,6] = -k717
  k_mat[17,17] = k918+k1618+k618+k1821
  k_mat[17,8] = -k918
  k_mat[17,15] = -k1618
  k_mat[17,20] = -k1821
  k_mat[18,18] = k719+k1920+k1719+k819
  k_mat[18,7] = -k819
  k_mat[18,6] = -k719
  k_mat[18,16] = -k1719
  k_mat[18,19] = -k1920
  k_mat[19,19] = k1920+k820+k2021+k720+k920
  k_mat[19,7] = -k820
  k_mat[19,6] = -k720
  k_mat[19,8] = -k920
  k_mat[19,20] = -k2021
  k_mat[19,18] = -k1920
  k_mat[20,20] = k2021+k921+k1821+k821
  k_mat[20,7] = -k821
  k_mat[20,17] = -k1821
  k_mat[20,19] = -k2021
  k_mat[20,8] = -k921


  # T_input = my_data3[:,1:23]

  # T_reservoir = T_reservoir*np.ones((21,1))
  Temp = copy.deepcopy(T)
  T_ambient = T_ambient*np.ones((21,1))
  T_cool_arr = T
  T_temp_cool = 0
  error=0

  for x in range(len(time)-1):
    i = x+1
    heating_node_no = T_input[x,-1]
    heat_one_hot_vec = np.zeros((21,1))
    if heating_node_no != 0:
      heat_one_hot_vec[int(heating_node_no)-1] = 1
      neighbors = get_neighbors(heating_node_no)
      for neighbor in neighbors:
        heat_one_hot_vec[neighbor-1] = q_k
    # k_heat_terms = k_heat*(time[i])
    # heating_mat = np.exp(-k_heat_terms)*heat_one_hot_vec

    # k_terms = ke*np.ones((21,1))*(time[i])
    # convection_mat = np.exp(-k_terms)


    # T_heat=(-k_heat_terms*(T - T_reservoir)*heating_mat)  #Defining heating nodes
    # T_cool = (-k_terms*(T - T_ambient)*convection_mat)    #Defining cooling nodes
    # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T + T_cool*(time[i]-time[i-1]) + T_heat*(time[i]-time[i-1]) #Predicted Temperature
    # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) - ((k_heat*(T - T_reservoir))*heat_one_hot_vec*(time[i]-time[i-1]))
    T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) + k_heat*heat_one_hot_vec
    # T= -np.matmul(k_mat,T)*(time[i]-time[i-1]) + T - (ke*(T - T_ambient))*(time[i]-time[i-1]) + k_heat
    ke######################DONT FORGET TO READD K
    # print(np.shape(T[0]))
    Temp = np.concatenate((Temp,T),1)
    # print(Temp[8,:])
  # error_mat = np.array(abs(np.transpose(Temp)-T_input[:,0:21]))
  # error_mat = np.square(error_mat)
  # error+=error_mat.mean()
  print(Temp[8,:])

  return Temp

In [ ]:
# bounds = [(0, .3) for n in range(len(init_guess))]
# bounds[-1]=(0,.1)
# init_guess = [k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,Q9]
# results = minimize(thermal_func,init_guess)


In [ ]:
bnds = [(0, .5) for n in range(len(init_guess))]
for n in range(9):
  bnds[-n-2] = (0,None)
print(bnds[-10:-1])
bnds[-1]=(0,1)
bnds[-2]=(0,None)
results = minimize(unified_thermal_multi_q,init_guess,args,bounds=bnds)
# results = least_squares(unified_thermal_func_latent_node,init_guess, jac = '2-point',args = args,bounds=bnds)
print(results)


[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]
  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: 432.3091072967037
        x: [ 3.152e-02  9.625e-02 ...  8.691e+01  2.821e-01]
      nit: 161
      jac: [-1.272e+01 -7.622e+00 ... -9.707e-01 -4.579e+01]
     nfev: 15066
     njev: 162
 hess_inv: <92x92 LbfgsInvHessProduct with dtype=float64>


In [ ]:
k12 = 0.147
k23 = .145
k14 = .158
k25 = .21
k36 = .164
k45 = .175
k56 = .115
k47 = .203
k58 = .12166
k69 = .176
k78 = .167
k89 = .168
k110 = 0.0537
k113 = 0.0122
k211 = 0.0283
k312 = 0.0744
k314 = 0.0917
k415 = 0.0449
k616 = 0.105
k717 = 0.0715
k719 = 0.17
k820 = 0.0107
k918 = 0.126
k921 = 0.1738
k1011 = 0.077
k1112 = 0.088
k1315 = 0.109
k1416 = 0.0435
k1517 = 0.02
k1618 = 0.068
k1920 = 0.1577
k2021 = 0.1337
ke1 = .0
ke2 = .0
ke3 = .0
ke4 = .12668
ke5 = .12644
ke6= .0
ke7 = .0536
ke8 = .0
ke9 = .0
ke10 = .0059
ke11 = .0
ke12 = .00303
ke13 = .0
ke14 = .000279
ke15 = .01795
ke16 = .0
ke17 = .0172
ke18 = .00316
ke19 = 0.0874
ke20 = 0.0807
ke21 = 0.02414
kh1 = 0.000694
kh2 = 0.07577
kh3 = 0.0014
kh4 = 0.0212
kh5 = 0.0222
kh6 = 0.01
kh7 = 0.0256
kh8 = 0.00312
kh9 =0.0424

input = copy.deepcopy(np.asarray(results.x))
input
# input = np.array([5.71271322e-02, 3.22634140e-02, 7.45668689e-02, 1.09564924e-01,
#        3.84051284e-01, 7.47987872e-02, 3.35687390e-02, 7.67750146e-02,
#        1.70628317e-02, 4.60519235e-02, 5.99724926e-02, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
#        1.75990963e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
#        5.19523610e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 2.98141254e-01, 4.01287659e-03, 2.70288547e-01,
#        1.69022944e-01, 2.67906692e-01, 1.52771192e-01, 0.00000000e+00,
#        0.00000000e+00, 2.10630483e-02, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.33361602e-03,
#        0.00000000e+00, 8.66853302e-03, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.41212018e-02,
#        6.53778574e-04, 2.17859163e-03, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.06779975e-01,
#        1.65259309e-01, 0.00000000e+00, 0.00000000e+00, 1.03317103e-01,
#        1.51578013e-02, 4.88330462e-02, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.19545136e-01,
#        0.00000000e+00, 5.25683925e-02, 7.00122894e-02, 3.11129204e-02,
#        4.55448391e-02, 4.13617544e-02, 2.95081092e-02, 0.00000000e+00,
#        1.72020516e-01, 4.71450778e-02, 2.39453649e-02, 1.72409990e-01,
#        4.40929971e-02, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
#        1.00000000e+00, 1.00000000e+00, 5.00000000e-02, 1.00000000e+00,
#        3.99996386e-01, 1.00000000e+00, 9.24987563e+01])
# print(input)
# input[-1] = 300
# input[-20:-25]=0
# input[-6] = 1.5
# print(input)
# Q9 = 0
# T_ambient = 92.8
# T_reservoir = 300.069
# input = [k12,k23,k14,k25,k36,k45,k56,k47,k58,k69,k78,k89,k110,k113,k211,k312,k314,k415,k616,k717,k719,k820,k918,k921,k1011,k1112,k1315,k1416,k1517,k1618,k1920,k2021,ke1,ke2,ke3,ke4,ke5,ke6,ke7,ke8,ke9,ke10,ke11,ke12,ke13,ke14,ke15,ke16,ke17,ke18,ke19,ke20,ke21,kh1,kh2,kh3,kh4,kh5,kh6,kh7,kh8,kh9,T_ambient,T_reservoir]
# time_1 = [0 +.1*interval for interval in range(500)]
print(time)
temperature = unified_thermal_multi_q_plot(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input)
print(len(temperature[1,:]))


[ 0.93444157  1.06231022  1.15825558  1.28646326  1.41433334  1.51054239
  1.63813686  1.73449326  1.86229873  1.95849299  2.08614206  2.2147038
  2.31027484  2.43825817  2.53419709  2.66289449  2.75855422  2.88640141
  2.98210549  3.11044431  3.23817635  3.33417773  3.46216273  3.5584991
  3.68621063  3.7826755   3.91014814  4.03859615  4.13409567  4.26242042
  4.35861254  4.48656607  4.58225322  4.71015811  4.80623007  4.93447161
  5.06222367  5.15837097  5.28618836  5.38253188  5.51016974  5.60641646
  5.73424292  5.83091497  5.95845652  6.08629608  6.18243074  6.31076169
  6.40617585  6.53418755  6.63029623  6.75874162  6.8862617   6.98225474
  7.11041856  7.20653915  7.33417749  7.43031287  7.55823708  7.65481138
  7.78205347  7.9101584   8.00628161  8.13452315  8.2300992   8.3583405
  8.45419168  8.58292103  8.71024013  8.80633259  8.93410897  9.03046918
  9.15833449  9.25437236  9.3825388   9.47850251  9.60625219  9.73424983
  9.83018041  9.9588294  10.05417371 10.18222427 10.27

TypeError: ignored

In [ ]:
print(results.x[0:32])
print(results.x[0:21])
print(results.x[0:9])

# input = np.array([5.71271322e-02, 3.22634140e-02, 7.45668689e-02, 1.09564924e-01,
#        3.84051284e-01, 7.47987872e-02, 3.35687390e-02, 7.67750146e-02,
#        1.70628317e-02, 4.60519235e-02, 5.99724926e-02, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
#        1.75990963e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
#        5.19523610e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 2.98141254e-01, 4.01287659e-03, 2.70288547e-01,
#        1.69022944e-01, 2.67906692e-01, 1.52771192e-01, 0.00000000e+00,
#        0.00000000e+00, 2.10630483e-02, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.33361602e-03,
#        0.00000000e+00, 8.66853302e-03, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.41212018e-02,
#        6.53778574e-04, 2.17859163e-03, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.06779975e-01,
#        1.65259309e-01, 0.00000000e+00, 0.00000000e+00, 1.03317103e-01,
#        1.51578013e-02, 4.88330462e-02, 0.00000000e+00, 0.00000000e+00,
#        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.19545136e-01,
#        0.00000000e+00, 5.25683925e-02, 7.00122894e-02, 3.11129204e-02,
#        4.55448391e-02, 4.13617544e-02, 2.95081092e-02, 0.00000000e+00,
#        1.72020516e-01, 4.71450778e-02, 2.39453649e-02, 1.72409990e-01,
#        4.40929971e-02, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
#        1.00000000e+00, 1.00000000e+00, 5.00000000e-02, 1.00000000e+00,
#        3.99996386e-01, 1.00000000e+00, 9.24987563e+01])

In [ ]:
def compare_plot_sim_data(data,input):
  if data == "my_data1":

    time = my_data1[:,0]
    T1=my_data1[0,1]
    T2=my_data1[0,2]
    T3=my_data1[0,3]
    T4=my_data1[0,4]
    T5=my_data1[0,5]
    T6=my_data1[0,6]
    T7=my_data1[0,7]
    T8=my_data1[0,8]
    T9=my_data1[0,9]
    T10=my_data1[0,10]
    T11=my_data1[0,11]
    T12=my_data1[0,12]
    T13=my_data1[0,13]
    T14=my_data1[0,14]
    T15=my_data1[0,15]
    T16=my_data1[0,16]
    T17=my_data1[0,17]
    T18=my_data1[0,18]
    T19=my_data1[0,19]
    T20=my_data1[0,20]
    T21=my_data1[0,21]
    heated_node_id=my_data1[0,22]
    T_input = my_data1[:,1:23]
    plot_data = my_data1

  elif data == "my_data2":

    time = my_data2[:,0]
    T1=my_data2[0,1]
    T2=my_data2[0,2]
    T3=my_data2[0,3]
    T4=my_data2[0,4]
    T5=my_data2[0,5]
    T6=my_data2[0,6]
    T7=my_data2[0,7]
    T8=my_data2[0,8]
    T9=my_data2[0,9]
    T10=my_data2[0,10]
    T11=my_data2[0,11]
    T12=my_data2[0,12]
    T13=my_data2[0,13]
    T14=my_data2[0,14]
    T15=my_data2[0,15]
    T16=my_data2[0,16]
    T17=my_data2[0,17]
    T18=my_data2[0,18]
    T19=my_data2[0,19]
    T20=my_data2[0,20]
    T21=my_data2[0,21]
    heated_node_id=my_data2[0,22]
    T_input = my_data2[:,1:23]
    plot_data = my_data2

  elif data == "my_data3":
    time = my_data3[:,0]
    T1=my_data3[0,1]
    T2=my_data3[0,2]
    T3=my_data3[0,3]
    T4=my_data3[0,4]
    T5=my_data3[0,5]
    T6=my_data3[0,6]
    T7=my_data3[0,7]
    T8=my_data3[0,8]
    T9=my_data3[0,9]
    T10=my_data3[0,10]
    T11=my_data3[0,11]
    T12=my_data3[0,12]
    T13=my_data3[0,13]
    T14=my_data3[0,14]
    T15=my_data3[0,15]
    T16=my_data3[0,16]
    T17=my_data3[0,17]
    T18=my_data3[0,18]
    T19=my_data3[0,19]
    T20=my_data3[0,20]
    T21=my_data3[0,21]
    heated_node_id=my_data3[0,22]
    T_input = my_data3[:,1:23]
    plot_data = my_data3

  elif data == "my_data4":
    time = my_data4[:,0]
    T1=my_data4[0,1]
    T2=my_data4[0,2]
    T3=my_data4[0,3]
    T4=my_data4[0,4]
    T5=my_data4[0,5]
    T6=my_data4[0,6]
    T7=my_data4[0,7]
    T8=my_data4[0,8]
    T9=my_data4[0,9]
    T10=my_data4[0,10]
    T11=my_data4[0,11]
    T12=my_data4[0,12]
    T13=my_data4[0,13]
    T14=my_data4[0,14]
    T15=my_data4[0,15]
    T16=my_data4[0,16]
    T17=my_data4[0,17]
    T18=my_data4[0,18]
    T19=my_data4[0,19]
    T20=my_data4[0,20]
    T21=my_data4[0,21]
    heated_node_id=my_data4[0,22]
    T_input = my_data4[:,1:23]
    plot_data = my_data4
  elif data == "test_my_data1":
    time = test_my_data1[:,0]
    T1=test_my_data1[0,1]
    T2=test_my_data1[0,2]
    T3=test_my_data1[0,3]
    T4=test_my_data1[0,4]
    T5=test_my_data1[0,5]
    T6=test_my_data1[0,6]
    T7=test_my_data1[0,7]
    T8=test_my_data1[0,8]
    T9=test_my_data1[0,9]
    T10=test_my_data1[0,10]
    T11=test_my_data1[0,11]
    T12=test_my_data1[0,12]
    T13=test_my_data1[0,13]
    T14=test_my_data1[0,14]
    T15=test_my_data1[0,15]
    T16=test_my_data1[0,16]
    T17=test_my_data1[0,17]
    T18=test_my_data1[0,18]
    T19=test_my_data1[0,19]
    T20=test_my_data1[0,20]
    T21=test_my_data1[0,21]
    heated_node_id=test_my_data1[0,22]
    T_input = test_my_data1[:,1:23]
    plot_data = test_my_data1
  elif data == "test_my_data2":
    time = test_my_data2[:,0]
    T1=test_my_data2[0,1]
    T2=test_my_data2[0,2]
    T3=test_my_data2[0,3]
    T4=test_my_data2[0,4]
    T5=test_my_data2[0,5]
    T6=test_my_data2[0,6]
    T7=test_my_data2[0,7]
    T8=test_my_data2[0,8]
    T9=test_my_data2[0,9]
    T10=test_my_data2[0,10]
    T11=test_my_data2[0,11]
    T12=test_my_data2[0,12]
    T13=test_my_data2[0,13]
    T14=test_my_data2[0,14]
    T15=test_my_data2[0,15]
    T16=test_my_data2[0,16]
    T17=test_my_data2[0,17]
    T18=test_my_data2[0,18]
    T19=test_my_data2[0,19]
    T20=test_my_data2[0,20]
    T21=test_my_data2[0,21]
    heated_node_id=test_my_data2[0,22]
    T_input = test_my_data2[:,1:23]
    plot_data = test_my_data2
  elif data == "test_my_data3":
    time = test_my_data3[:,0]
    T1=test_my_data3[0,1]
    T2=test_my_data3[0,2]
    T3=test_my_data3[0,3]
    T4=test_my_data3[0,4]
    T5=test_my_data3[0,5]
    T6=test_my_data3[0,6]
    T7=test_my_data3[0,7]
    T8=test_my_data3[0,8]
    T9=test_my_data3[0,9]
    T10=test_my_data3[0,10]
    T11=test_my_data3[0,11]
    T12=test_my_data3[0,12]
    T13=test_my_data3[0,13]
    T14=test_my_data3[0,14]
    T15=test_my_data3[0,15]
    T16=test_my_data3[0,16]
    T17=test_my_data3[0,17]
    T18=test_my_data3[0,18]
    T19=test_my_data3[0,19]
    T20=test_my_data3[0,20]
    T21=test_my_data3[0,21]
    heated_node_id=test_my_data3[0,22]
    T_input = test_my_data3[:,1:23]
    plot_data = test_my_data3

  args=(time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input)
  temperature = unified_thermal_multi_q_plot(input,time,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,T11,T12,T13,T14,T15,T16,T17,T18,T19,T20,T21,T_input)

  figure, axes = plt.subplots(nrows=1, ncols=2)
  figure.tight_layout(pad=3.0)
  plt.subplot(1, 2, 1)
  plt.title('Predicted Node \nTemperature Plot')
  plt.ylabel(' Temperature (F)')
  plt.xlabel('Time (t)')
  plt.xlim(0, 100)
  plt.ylim(80, 160)
  plt.plot(np.transpose(temperature[:,:]))
  plt.subplot(1, 2, 2)
  plt.title('Actual Node \nTemperature Plot')
  plt.ylabel(' Temperature (F)')
  plt.xlabel('Time (t)')

  plt.xlim(0, 100)
  plt.ylim(80, 160)
  plt.plot(plot_data[:,1:21])
  plt.savefig('Plot1.png')


In [ ]:
# plt.plot(np.transpose(temperature[4,:]))
figure, axes = plt.subplots(nrows=1, ncols=2)
figure.tight_layout(pad=3.0)
plt.subplot(1, 2, 1)
plt.title('Predicted Node \nTemperature Plot')
plt.ylabel(' Temperature (F)')
plt.xlabel('Time (t)')
plt.plot(np.transpose(temperature[:,:]))
plt.subplot(1, 2, 2)
plt.title('Actual Node \nTemperature Plot')
plt.ylabel(' Temperature (F)')
plt.xlabel('Time (t)')
plt.plot(my_data3[:,1:21])
plt.savefig('Plot1.png')


In [ ]:
compare_plot_sim_data("my_data1",input)
compare_plot_sim_data("my_data2",input)
compare_plot_sim_data("my_data3",input)
compare_plot_sim_data("my_data4",input)

compare_plot_sim_data("test_my_data1",input)
compare_plot_sim_data("test_my_data2",input)
compare_plot_sim_data("test_my_data3",input)
